# Final Capstone Project
## Best location for a restaurant among top 12 cities worldwide

#### by Ignacio de Juan
#### November 2020, Final Project Assignment for IBM Data Science Course

### Introduction / Business Problem

We need to take the decision of where is the best location to open a new restaurant of a certain cuisine (e.g. Italian, Japanese, Spanish, etc). Which are the cities with a highest potential based on the level of concentration of certain cuisine and the demand of each cuisine based also on the assessment of the existing restaurants? If possible, provide with a recommendation for the location of the restaurant and the cuisine that would be likely to be more successful.

### Data

We will take data from 12 top cities worldwide.

The criteria to select those cities is to choose among cities that have a similar GDP per capita and a large enough base of restaurants so that the study is statistically significant.

We will be using Wikipedia to identify the target cities of our study and Foursquare as our database of venue data.